In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import os
# import json
# import torch
# from PIL import Image
# from torch.utils.data import Dataset, DataLoader
# from transformers import BertTokenizer, BertModel, CLIPProcessor, CLIPVisionModel

# # Paths of uploaded dataset files
# IMG_DIR = "/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images"
# QUEST_PATH = "/kaggle/input/vqatext/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json"
# ANNOT_PATH = "/kaggle/input/vqatext/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json"

# # using GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Verified Device: {device}")

# # # Paths based on your uploaded Kaggle screenshot
# # IMG_DIR = "/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images"
# # QUEST_PATH = "/kaggle/input/vqatext/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json"
# # ANNOT_PATH = "/kaggle/input/vqatext/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json"

In [ ]:

# class MiniVQADataset(Dataset):
#     def __init__(self, questions_path, annotations_path, img_dir, tokenizer, processor):
#         # 1. Load all data
#         with open(questions_path, 'r') as f:
#             all_questions = json.load(f)['questions']
#         with open(annotations_path, 'r') as f:
#             self.annotations = {a['question_id']: a for a in json.load(f)['annotations']}
        
#         self.img_dir = img_dir
#         self.tokenizer = tokenizer
#         self.processor = processor
#         # self.ans_to_id = {"yes": 0, "no": 1, "2": 2, "1": 3, "white": 4}

#         # --- NEW CODE STARTS HERE: Create a dynamic Answer Vocabulary ---
#         # Get all answers from the annotations you actually have
#         all_raw_answers = [a['multiple_choice_answer'] for a in self.annotations.values()]
        
#         # Get the 1000 most frequent answers (this is standard VQA research practice)
#         most_common = Counter(all_raw_answers).most_common(1000)
#         self.ans_to_id = {ans[0]: i for i, ans in enumerate(most_common)}
        

#         # 2. FILTERING LOGIC: Only keep questions if the image exists on disk
#         self.valid_questions = []
#         print("Filtering dataset for existing images... please wait.")
        
#         for q in all_questions:
#             img_id = str(q['image_id']).zfill(12)
#             img_path = os.path.join(self.img_dir, f"COCO_train2014_{img_id}.jpg")
            
#             if os.path.exists(img_path):
#                 self.valid_questions.append(q)
            
#             # Stop once we have 2000 valid samples to save time
#             if len(self.valid_questions) >= 10000:
#                 break
                
#         print(f"Done! Found {len(self.valid_questions)} matching images in your mini dataset.")

#     def __len__(self):
#         return len(self.valid_questions)

#     def __getitem__(self, idx):
#         q_data = self.valid_questions[idx]
#         q_id = q_data['question_id']
#         img_id = str(q_data['image_id']).zfill(12)
        
#         img_path = os.path.join(self.img_dir, f"COCO_train2014_{img_id}.jpg")
#         image = Image.open(img_path).convert("RGB")
        
#         text_inputs = self.tokenizer(q_data['question'], padding='max_length', max_length=32, truncation=True, return_tensors="pt")
#         image_inputs = self.processor(images=image, return_tensors="pt")
        
#         ans_str = self.annotations[q_id]['multiple_choice_answer']
#         label = self.ans_to_id.get(ans_str, 0) 

#         return {
#             'input_ids': text_inputs['input_ids'].squeeze(),
#             'attention_mask': text_inputs['attention_mask'].squeeze(),
#             'pixel_values': image_inputs['pixel_values'].squeeze(),
#             'label': torch.tensor(label)
#         }

In [ ]:
# class VQAModel(torch.nn.Module):
#     def __init__(self, num_answers=1000):
#         super().__init__()
#         self.bert = BertModel.from_pretrained("bert-base-uncased")
#         self.clip = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
        
#         # BERT (768) + CLIP (768) = 1536 features
#         self.classifier = torch.nn.Sequential(
#             torch.nn.Linear(1536, 512),
#             torch.nn.ReLU(),
#             torch.nn.Linear(512, num_answers)
#         )

#     def forward(self, input_ids, attention_mask, pixel_values):
#         # Extract Text Features
#         text_features = self.bert(input_ids, attention_mask=attention_mask).pooler_output
#         # Extract Image Features
#         image_features = self.clip(pixel_values).pooler_output
#         # Fuse and Classify
#         fused = torch.cat((text_features, image_features), dim=1)
#         return self.classifier(fused)

In [ ]:
# import torch
# from torch.utils.data import DataLoader, random_split
# from transformers import BertTokenizer, CLIPProcessor
# from tqdm import tqdm  # This adds a visual progress bar
# from collections import Counter


# # 1. Initialize Components
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


# # 1. Initialize with 3000 samples
# # This will look through your JSON and stop once it finds 3000 images on your disk
# dataset_full = MiniVQADataset(QUEST_PATH, ANNOT_PATH, IMG_DIR, tokenizer, processor)
# # Inside your class, change 'if len(self.valid_questions) >= 2000' to 3000

# # 2. Define the split (80% for training, 20% for testing)
# train_count = int(0.8 * len(dataset_full)) # 2400 samples
# test_count = len(dataset_full) - train_count # 600 samples

# # 3. Perform the split
# # random_split gives you two subsets of your data
# train_ds, test_ds = random_split(dataset_full, [train_count, test_count])



# # Create Dataset and Loader
# # train_ds = MiniVQADataset(QUEST_PATH, ANNOT_PATH, IMG_DIR, tokenizer, processor)
# # loader = DataLoader(train_ds, batch_size=8, shuffle=True)
# # 4. Create separate DataLoaders
# # The model only "sees" the train_loader during training
# train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

# # 2. Setup Model & Optimizer
# # model = VQAModel(num_answers=1000).to(device) # num_answers should match your class output

# # 2. Initialize Model with your new 1000-answer limit
# model = VQAModel(num_answers=1000)

# # 3. ACTIVATE BOTH GPUs
# if torch.cuda.device_count() > 1:
#     print(f"🔥 Using {torch.cuda.device_count()} GPUs for training!")
#     model = torch.nn.DataParallel(model)

# # 4. Move the model to the device
# model.to(device)

# # opt = torch.optim.AdamW(model.parameters(), lr=5e-5)
# opt = torch.optim.AdamW(model.parameters(), lr=1e-4)
# loss_fn = torch.nn.CrossEntropyLoss()

# # 3. Merged Training Loop with Accuracy
# # NUM_EPOCHS = 1 # Start with 1 to test
# NUM_EPOCHS = 40 # Start with 1 to test
# model.train()

# for epoch in range(NUM_EPOCHS):
#     running_corrects = 0
#     total_samples = 0
    
#     # tqdm creates a progress bar so you don't just see a static number
#     progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    
#     for batch in progress_bar:
#         # Move batch to GPU
#         ids = batch['input_ids'].to(device)
#         mask = batch['attention_mask'].to(device)
#         pixels = batch['pixel_values'].to(device)
#         labels = batch['label'].to(device)
        
#         # Forward Pass
#         # opt.zero_grad()
#         opt.zero_grad(set_to_none=True)
#         outputs = model(ids, mask, pixels)
#         loss = loss_fn(outputs, labels)
        
#         # --- Accuracy Calculation ---
#         _, preds = torch.max(outputs, 1)            # Get the predicted answer ID
#         running_corrects += (preds == labels).sum().item() # Count matches
#         total_samples += labels.size(0)             # Count total items
#         current_acc = 100 * running_corrects / total_samples
#         # ----------------------------

#         # Backward Pass
#         loss.backward()
#         opt.step()
        
#         # Update progress bar with Loss and Accuracy
#         progress_bar.set_postfix({
#             'loss': f'{loss.item():.4f}', 
#             'acc': f'{current_acc:.2f}%'
#         })

# # 4. Save Final Weights
# # torch.save(model.state_dict(), "/kaggle/working/vqa_mini.pth")
# # after using 2 GPUs
# # Use .module to get the original model inside the wrapper
# torch.save(model.module.state_dict(), "/kaggle/working/vqa_mini.pth")

# print(f"\nTraining Finished! Final Accuracy: {current_acc:.2f}%")

In [ ]:
# # Save the weights safely
# if isinstance(model, torch.nn.DataParallel):
#     # This reaches inside the multi-GPU wrapper to get the actual weights
#     torch.save(model.module.state_dict(), "/kaggle/working/vqa_best_97.pth")
#     print("Multi-GPU weights saved successfully!")
# else:
#     torch.save(model.state_dict(), "/kaggle/working/vqa_best_97.pth")
#     print("Single-GPU weights saved successfully!")

# Testing 

In [ ]:
# model.eval()  # Set model to evaluation mode
# test_running_corrects = 0
# test_total_samples = 0

# print("Starting Evaluation on Test Set...")

# # Disable gradient calculations for speed and memory efficiency
# with torch.no_grad():
#     # Use tqdm to monitor progress on the 600 test samples
#     test_progress_bar = tqdm(test_loader, desc="Testing")
    
#     for batch in test_progress_bar:
#         ids = batch['input_ids'].to(device)
#         mask = batch['attention_mask'].to(device)
#         pixels = batch['pixel_values'].to(device)
#         labels = batch['label'].to(device)

#         # Forward Pass
#         outputs = model(ids, mask, pixels)
        
#         # Calculate Accuracy
#         _, preds = torch.max(outputs, 1)
#         test_running_corrects += (preds == labels).sum().item()
#         test_total_samples += labels.size(0)
        
#         # Update progress bar
#         current_test_acc = 100 * test_running_corrects / test_total_samples
#         test_progress_bar.set_postfix({'test_acc': f'{current_test_acc:.2f}%'})

# final_test_accuracy = 100 * test_running_corrects / test_total_samples
# print(f"\n--- Evaluation Results ---")
# print(f"Final Test Accuracy: {final_test_accuracy:.2f}%")
# print(f"Training Accuracy was: {current_acc:.2f}%") # This is from your training loop
# print(f"The 'Generalization Gap' is: {current_acc - final_test_accuracy:.2f}%")

In [ ]:
# model.eval()
# test_running_corrects = 0
# test_total_samples = 0

# print(f"Starting Evaluation on {len(test_ds)} images...")

# with torch.no_grad():
#     # We set total=len(test_ds) and update by the batch size manually
#     test_progress_bar = tqdm(total=len(test_ds), desc="Testing Images")
    
#     for batch in test_loader:
#         ids = batch['input_ids'].to(device)
#         mask = batch['attention_mask'].to(device)
#         pixels = batch['pixel_values'].to(device)
#         labels = batch['label'].to(device)

#         outputs = model(ids, mask, pixels)
        
#         _, preds = torch.max(outputs, 1)
#         batch_corrects = (preds == labels).sum().item()
#         test_running_corrects += batch_corrects
#         test_total_samples += labels.size(0)
        
#         # Update bar by number of images in this batch (usually 8)
#         test_progress_bar.update(labels.size(0))
        
#         current_test_acc = 100 * test_running_corrects / test_total_samples
#         test_progress_bar.set_postfix({'test_acc': f'{current_test_acc:.2f}%'})

#     test_progress_bar.close()

# print(f"\nFinal Test Accuracy: {current_test_acc:.2f}%")

# Testing on Real image

In [ ]:
# import torch
# from PIL import Image
# import torchvision.transforms as transforms
# from transformers import BertTokenizer

# # --- CONFIGURATION ---
# # Use the same variable name you used during training!
# # If your model variable is 'vqa_net', change 'model' to 'vqa_net' below.
# my_model = model  

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# TOKENIZER = BertTokenizer.from_pretrained('bert-base-uncased')
# IMG_SIZE = (224, 224) 
# MAX_LEN = 32           

# # --- 1. DYNAMIC ANSWER MAPPING ---
# # This uses your existing label_encoder to handle all 1000+ classes
# def decode_answer(idx):
#     # 'label_encoder' must be the one you used to fit your training data
#     return label_encoder.inverse_transform([idx])[0]

# # --- 2. PREPROCESSING ---
# def preprocess(image_path, question):
#     img = Image.open(image_path).convert('RGB')
#     transform = transforms.Compose([
#         transforms.Resize(IMG_SIZE),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])
#     img_tensor = transform(img).unsqueeze(0).to(DEVICE)

#     tokens = TOKENIZER(
#         question,
#         padding='max_length',
#         truncation=True,
#         max_length=MAX_LEN,
#         return_tensors="pt"
#     ).to(DEVICE)
    
#     return img_tensor, tokens['input_ids'], tokens['attention_mask']

# # --- 3. RUN PREDICTION ---
# def get_answer(img_path, q_text):
#     img_t, input_ids, att_mask = preprocess(img_path, q_text)
    
#     my_model.eval() 
#     with torch.no_grad():
#         # Ensure the arguments (img, ids, mask) match your model's forward() order
#         output = my_model(img_t, input_ids, att_mask) 
#         idx = torch.argmax(output, dim=1).item()
        
#     prediction = decode_answer(idx)
#     print(f"Question: {q_text}")
#     print(f"Predicted Answer: {prediction}")

# # --- TEST ---
# get_answer("/kaggle/input/doggyy/dog.jpg", "tell me the animal shown in image")

In [3]:
import json
import torch
import time
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from torch import nn
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms
from collections import Counter
from tqdm import tqdm

import numpy as np
import pandas as pd

import os
# set cuda
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [5]:
# Paths of uploaded dataset files
IMG_DIR = "/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images"
QUEST_PATH = "/kaggle/input/vqatext/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json"
ANNOT_PATH = "/kaggle/input/vqatext/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json"


In [6]:
# Image pre-processing : to properly utilize ResNET50 (ImageNet)
val_transform = transforms.Compose([
    transforms.Resize(256),          # Resize shorter side to 256
    transforms.CenterCrop(224),      # crop center to 224X224
    transforms.ToTensor(),           # convert to Tensor from PIL object (Python Image loader)
    transforms.Normalize(
        mean = [0.485, 0.456, 0.406],
        std  = [0.229, 0.224, 0.225]
    )
])

# Training transform - Data Augmentation 
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),    # Random crop + resize to 224X224
    transforms.RandomHorizontalFlip(),    # randomly flip horizontally
    transforms.ColorJitter(brightness=0.2, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = [0.485, 0.456, 0.406],
        std  = [0.229, 0.224, 0.225]
    )
])

In [7]:
class ImageEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)   # Loading a RestNet50 pretrained on ImageNET (IMAGENET1K_V2 is a strong, modern weight set)

    # Remove last classification layer[2048->1000] and replace it with [2048->512]
    self.resnet.fc = nn.Linear(2048, 512)

    # Freeze all resnet parameters except except last fc (fully-connected) layer
    for name, param in self.resnet.named_parameters():
      if not name.startswith("fc"):
        param.requires_grad = False

  # Forward pass
  def forward(self, x):
    return self.resnet(x)    # [B, 512]
  # Input: x is batch of images shaped [B, 3, 224, 224], preprocessed with transforms (resize, crop, normalize)
  # Output: a batch of embeddings [B, 512] from the modified ResNet50
  # Reason:  this vector is visual reprstn you'll feed into VQA model, often concatenated, attended over, or fused with text embeddings           


In [9]:
class TextEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = BertModel.from_pretrained("bert-base-uncased")  # all text is lowercased 
    # Projection layer
    self.fc = nn.Linear(768, 512)  # Adds a fc(linear) layer to project BERT's output dim.(768) down to 512

    # Freeze all BERT parameters > they won't be updated during training
    for param in self.bert.parameter():
      param.requires_grad = False
    # - Only the projection layer (fc) learns to adapt BERT’s embeddings to your VQA task.

    # Forward pass
    def forward(self, input_ids, attention_mask):
      output = self.bert(
          input_ids = input_ids,
          attention_mask = attention_mask
      )    # output: contains hidden states for each token & pooled outputs

      # CLS token extraction 
      cls_embedding = output.last_hidden_state[:, 0, :]  # [B, 768]-> one vector per q.
      cls_embedding = self.fc(cls_embedding)   # [B, 512]   projects the CLS embeddings down to 512 dimensions 
      
      return cls_embedding   
      # - This vector represents the semantic meaning of the question, ready to be combined with the image embedding.



In [11]:
class VQADataset(Dataset):
  def __init__(self, image_dir, questions_path, annotations_path, tokenizer, answer_to_idx, transform):

    self.image_dir = image_dir
    self.transform = transform
    self.tokenizer = tokenizer
    self.answer_to_idx = answer_to_idx

    with open(questions_path, 'r') as f:
      self.questions = json.load(f)["questions"]
    with open(annotations_path, 'r') as f:
      anns = json.load(f)["annotations"]

    self.ann_map = {ann["questions_id"]: ann for ann in anns}  # building a dictionary mapping q_id -> annotations, making it easy to lookup for answer for a given q.
  
  def __len__(self):
    return len(self.questions)    # needed by Pytorch's dataloader

  def __getitem__(self, idx):
    q = self.questions[idx]
    qid = q["questions_id"]

    # Image loading
    img_name = f"COCO_train2014_{q['image_id']:012d}.jpg"   # :012d = zero-pads the ID to 12 digits (COCO naming convention)
    img_path = os.path.join(self.image_dir, img_name)
    image = Image.open(img_path).convert("RGB")
    image = self.transform(image)
    # Constructing the full path, opening image using PIL -> converts to RGB, then applying pre-processing transforms


    # Question Tokenization 
    tokens = self.tokenizer(
        q["question"], 
        padding = "max_length",
        truncation = True,
        max_length = 32,
        return_tensors = "pt"
    ) 
    # tokenizes the qs. using BERT tokenizer, pads/truncates to fixed lengths (32 tokens), returns PyTorch tensors (input_ids, attention_mask)

    # Answer label 
    answer = self.ann_map[qid]['multiple_choice_answer']
    label = self.answer_to_idx.get(answer, -1)      

    
    # Return a tuple 
    return (
        image, 
        tokens['input_ids'].squueze(0),
        tokens['attention_mask'].squueze(0),
        torch.tensor(label)          
    )
    #   This tuple is exactly what the model needs: 
    # visual features + text features + answer label


In [13]:
class VQAModel(nn.Module):
  def __init__(self, num_answers):
    super().__init__()

    # Image Encoder 
    self.cnn = resnet50(pretrained=True)
    # Backbone: loads resnet50 pretrained on ImageNet for strong visual features
    self.cnn.fc = nn.Identity()
    # Head removal - replace the final classification layer with Identity - so the output is 2048-dim pooled feature instead of 1000 class-logits

    for p in self.cnn.parameters():
      p.requires_grad = False
    # Freezing: stopping all CNN parameters from updating - keeping pre-trained features fixed


    #   Text encoder (BERT) -
    self.bert = BertModel.from_pretrained("bert-base-uncased")
    # Loads BERT-base (uncased) for contextual q. embeddings 
    for layer in self.bert.encoder.layer[:8]:
      for p in layer.parameters():
        p.requires_grad = False
    # Partial freezing : Freezes the first 8 transformer layers; keeps the top 4 trainable,  - lower layer capture general semantics, upper layer adapts to task 


    #  FUSION and CLASSIFIER HEAD 
    self.classifier = nn.Sequential(
        nn.Linear(2048 + 768, 512),
        nn.ReLU(),
        nn.Dropout(0.5), 
        nn.Linear(512, num_answers)
    )
    # Projection: Linear(2816->512) compresses fused features
    # Activation: ReLU  to add non--linearity
    # Regularization: Dropout(0.5) to combat over-fitting
    # Output: Linear(512->num_answers) produces logit over answer vocabulary


  #  FORWARD  PASS
  def forward(self, image, input_ids, attention_mask):
    img_feat = self.cnn(image)
    #  Image features: Passes preprocessed images [B,3,224,224] through ResNet50 → global features [B,2048]
    
    txt_feat = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask
    ).last_hidden_state[:, 0, :]    # running BERT on tokenized questions

    fused = torch.cat([img_feat, txt_feat], dim=1)
    # fusion : concat. img & text features along feature dimension-> [B, 2816]
    return self.classifier(fused)  
    #  Prediction: Applies the classifier to produce logits [B, num_answers]
 

In [16]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()

    total_loss = 0.0
    correct = 0
    total = 0

    for batch in loader:
        images = batch["image"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        mask = labels != -1
        if mask.sum() == 0:
            continue

        outputs = model(images, input_ids, attention_mask)

        loss = criterion(outputs[mask], labels[mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 🔹 Training accuracy calculation
        preds = outputs.argmax(dim=1)
        correct += (preds[mask] == labels[mask]).sum().item()
        total += mask.sum().item()

    train_loss = total_loss / len(loader)
    train_acc = correct / total if total > 0 else 0

    return train_loss, train_acc

In [18]:
# needed bcz. the model needs a fixed set of possible answers (classes) to predict form
from collections import Counter
import json

TOP_K = 1000   # number of most frequent answers 
# Counter: a dictionary subclass that counts occurences of item
# TOP_K: defines how many of the most frequent answers you'll keep in your vocabulary (default=1000)

def build_answer_vocab(annotation_file, TOP_K):
  with open(annotation_file, 'r') as f:
    data = json.load(f)

  #  Collect answers 
  answers = []
  for ann in data['annotations']:
    # COCO VQA has 10 answers per question
    for ans in ann["answers"]:
      # normalize answer text 
      answer = ans['answer'].lower().strip()
      answers.append(answer)

  counter = Counter(answers)

  # take top-k most frequent answers
  most_common_answers = counter.most_common(TOP_K)

  answer_to_idx = {}
  idx_to_answer = {}

  for idx, (answer, _) in enumerate(most_common_answers):
    answer_to_idx[answer] = idx
    idx_to_answer[idx] = answer

  print(f"Total unique answers: {len(counter)}")
  print(f"Using top-{TOP_K} answers")

  return answer_to_idx, idx_to_answer


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_answers = 1500
model = VQAModel(num_answers).to(device)

# Loss fn / calculating criteria
criterion = torch.nn.CrossEntropyLoss()

# Optimizer - ADAM (used here)
optimizer = torch.optim.Adam(
    filter (lambda p: p.requires_grad, model.parameters()),
    lr = 2e-4
)

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size = 3,
    gamma = 0.3
)

# NEW FINAL MERGED GEM___ code

In [5]:
import json
import torch
import time
import os
import numpy as np
from PIL import Image
from collections import Counter
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm
import sys

# 1. Setup Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# 2. Define Transforms
# Image pre-processing : to properly utilize ResNET50 (ImageNet)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize directly to 224x224 for simplicity
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Device: cuda


In [9]:
# 3. Helper: Build Answer Vocabulary
def build_answer_vocab(annotations, top_k=1000):
    all_answers = []
    for ann in annotations:
        for ans in ann["answers"]:
            all_answers.append(ans['answer'].lower().strip())
    
    counter = Counter(all_answers)
    most_common = counter.most_common(top_k)
    
    answer_to_idx = {ans: i for i, (ans, _) in enumerate(most_common)}
    return answer_to_idx

# 4. Modified Dataset Class (Accepts data lists instead of paths)
class VQADataset(Dataset):
    def __init__(self, image_dir, questions_list, ann_map, tokenizer, answer_to_idx, transform):
        self.image_dir = image_dir
        self.questions = questions_list
        self.ann_map = ann_map
        self.tokenizer = tokenizer
        self.answer_to_idx = answer_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        q = self.questions[idx]
        qid = q["question_id"]
        
        # Image Loading
        img_name = f"COCO_train2014_{q['image_id']:012d}.jpg"
        img_path = os.path.join(self.image_dir, img_name)
        
        # Safety check if image exists (optional but good for debugging)
        try:
            image = Image.open(img_path).convert("RGB")
        except:
            # Create a dummy black image if file missing (prevents crash)
            image = Image.new('RGB', (224, 224))
            
        image = self.transform(image)

        # Tokenization
        tokens = self.tokenizer(
            q["question"],
            padding="max_length",
            truncation=True,
            max_length=32,
            return_tensors="pt"
        )

        # Label
        ann = self.ann_map.get(qid)
        label = -1
        if ann:
            # We take the 'multiple_choice_answer'
            ans_str = ann['multiple_choice_answer'].lower().strip()
            label = self.answer_to_idx.get(ans_str, -1)

        # Fixed typo: squueze -> squeeze
        return {
            "image": image,
            "input_ids": tokens['input_ids'].squeeze(0),
            "attention_mask": tokens['attention_mask'].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# 5. VQA Model (ResNet + BERT)
# class VQAModel(nn.Module):
#     def __init__(self, num_answers):
#         super().__init__()
        
#         # Image Encoder (ResNet50)
#         self.cnn = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
#         self.cnn.fc = nn.Identity() # Removes the last classification layer
#         for p in self.cnn.parameters():
#             p.requires_grad = False # Freeze CNN
            
#         # Text Encoder (BERT)
#         self.bert = BertModel.from_pretrained("bert-base-uncased")
#         for layer in self.bert.encoder.layer[:8]: # Freeze first 8 layers
#             for p in layer.parameters():
#                 p.requires_grad = False
                
#         # Classifier
#         self.classifier = nn.Sequential(
#             nn.Linear(2048 + 768, 512),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(512, num_answers)
#         )

#     def forward(self, image, input_ids, attention_mask):
#         img_feat = self.cnn(image)             # [B, 2048]
        
#         bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         txt_feat = bert_out.last_hidden_state[:, 0, :] # CLS token [B, 768]
        
#         fused = torch.cat([img_feat, txt_feat], dim=1) # [B, 2816]
#         logits = self.classifier(fused)
#         return logits

class VQAModel(nn.Module):
    def __init__(self, num_answers):
        super().__init__()
        
        # 1. Visual Encoder (ResNet50)
        self.cnn = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.cnn.fc = nn.Identity() 
        
        # UNFREEZE TRICK: Train the last block (layer4) to adapt to VQA
        for name, param in self.cnn.named_parameters():
            if "layer4" in name:
                param.requires_grad = True 
            else:
                param.requires_grad = False
            
        # 2. Text Encoder (BERT)
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        # Freeze first 8 layers, train last 4
        for layer in self.bert.encoder.layer[:8]: 
            for p in layer.parameters():
                p.requires_grad = False
                
        # 3. FUSION LAYERS (The Fix)
        # Project both to 1024 dim
        self.img_proj = nn.Linear(2048, 1024)
        self.txt_proj = nn.Linear(768, 1024)
        
        self.dropout = nn.Dropout(0.5)
        
        # 4. Classifier
        self.classifier = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_answers)
        )

    def forward(self, image, input_ids, attention_mask):
        # Image Features
        img_feat = self.cnn(image)         # [B, 2048]
        img_feat = self.img_proj(img_feat) # [B, 1024]
        
        # Text Features
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        txt_feat = bert_out.last_hidden_state[:, 0, :] # [B, 768]
        txt_feat = self.txt_proj(txt_feat)             # [B, 1024]
        
        # KEY CHANGE: Element-wise Multiplication
        fused = img_feat * txt_feat  
        
        fused = self.dropout(fused)
        logits = self.classifier(fused)
        return logits

In [12]:
# def train_one_epoch(model, loader, optimizer, criterion, device):
#     model.train()
#     total_loss = 0.0
#     correct = 0
#     total = 0

#     # TQDM wraps the loader to show a progress bar
#     # leave=False keeps the output clean (clears bar after epoch)
#     loop = tqdm(loader, desc="Training", leave=False)

#     for batch in loop:
#         images = batch["image"].to(device)
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["label"].to(device)

#         mask = labels != -1
#         if mask.sum() == 0:
#             continue

#         optimizer.zero_grad()
#         outputs = model(images, input_ids, attention_mask)
        
#         loss = criterion(outputs[mask], labels[mask])
        
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()
        
#         # Accuracy calculation
#         preds = outputs.argmax(dim=1)
#         correct += (preds[mask] == labels[mask]).sum().item()
#         total += mask.sum().item()

#         # Update the progress bar with current loss
#         loop.set_postfix(loss=loss.item())

#     avg_loss = total_loss / len(loader)
#     avg_acc = correct / total if total > 0 else 0
#     return avg_loss, avg_acc

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(loader, desc="Training", leave=False)
    
    for batch in loop:
        images = batch["image"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        mask = labels != -1
        if mask.sum() == 0: continue

        optimizer.zero_grad()
        outputs = model(images, input_ids, attention_mask)
        
        loss = criterion(outputs[mask], labels[mask])
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
        # Calculate running stats
        preds = outputs.argmax(dim=1)
        correct += (preds[mask] == labels[mask]).sum().item()
        total += mask.sum().item()
        
        # --- LIVE ACCURACY UPDATE ---
        # Calculate current average accuracy so far
        current_acc = correct / total if total > 0 else 0
        
        # Update the progress bar
        loop.set_postfix(loss=loss.item(), acc=f"{current_acc:.4f}")
        
    train_acc = correct / total if total > 0 else 0
    train_loss = total_loss / len(loader)
    
    return train_loss, train_acc


def validate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    
    # Add TQDM for validation as well
    loop = tqdm(loader, desc="Validating", leave=False)
    
    with torch.no_grad():
        for batch in loop:
            images = batch["image"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            mask = labels != -1
            if mask.sum() == 0:
                continue

            outputs = model(images, input_ids, attention_mask)
            preds = outputs.argmax(dim=1)
            
            correct += (preds[mask] == labels[mask]).sum().item()
            total += mask.sum().item()
            
    return correct / total if total > 0 else 0

In [ ]:
# --- CONFIGURATION ---
IMG_DIR = "/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images"
QUEST_PATH = "/kaggle/input/vqatext/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json"
ANNOT_PATH = "/kaggle/input/vqatext/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json"
TOP_K = 1000
BATCH_SIZE = 32

# 1. Load Data
print("Loading JSON data...")
with open(QUEST_PATH, 'r') as f:
    questions_json = json.load(f)
    all_questions = questions_json["questions"]

with open(ANNOT_PATH, 'r') as f:
    annotations_json = json.load(f)
    all_annotations = annotations_json["annotations"]

# Map question_id to annotation
ann_map = {ann["question_id"]: ann for ann in all_annotations}

# 2. Build Vocabulary
print("Building Vocabulary...")
answer_to_idx = build_answer_vocab(all_annotations, TOP_K)
num_classes = len(answer_to_idx)
print(f"Vocab size: {num_classes}")

# 3. Train/Test Split
# We split the list of questions, not the file paths
print("Splitting data...")
train_questions, val_questions = train_test_split(
    all_questions, 
    test_size=0.2, 
    random_state=42
)
print(f"Train size: {len(train_questions)}, Val size: {len(val_questions)}")

# 4. Create Datasets & Loaders
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_dataset = VQADataset(IMG_DIR, train_questions, ann_map, tokenizer, answer_to_idx, train_transform)
val_dataset = VQADataset(IMG_DIR, val_questions, ann_map, tokenizer, answer_to_idx, val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 5. Initialize Model
# model = VQAModel(num_answers=num_classes).to(device)

model = VQAModel(num_answers=num_classes) # Don't send to .to(device) yet

# Check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    # This wraps the model to split batches across both GPUs
    model = nn.DataParallel(model)

# Now send the model to the primary device (cuda:0)
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

# 6. Run Training
num_epochs = 6 # Reduced for testing, increase later
best_val_acc = 0.0

print("Starting training...")
# for epoch in range(num_epochs):
#     start_time = time.time()
    
#     train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
#     val_acc = validate(model, val_loader, device)
    
#     elapsed = time.time() - start_time
    
#     print(f"Epoch {epoch+1}/{num_epochs} | Time: {elapsed:.0f}s")
#     print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

#     torch.save(model.state_dict(), "vqa_last.pth")
    
#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         torch.save(model.state_dict(), "vqa_best.pth")
#         print("Saved best model.")

print("Starting training...")

for epoch in range(num_epochs):
    
    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    
    # Validate
    val_acc = validate(model, val_loader, device)
    
    # Clean print after the progress bars are done
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # torch.save(model.state_dict(), "vqa_last.pth")
    torch.save(model.module.state_dict(), "vqa_last.pth")
    print("saved last model")
    
    # Save checkpoints
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        # torch.save(model.state_dict(), "vqa_best.pth")
        torch.save(model.module.state_dict(), "vqa_best.pth")
        print(" -> Saved best model.")
    
    !zip model.zip vqa_best.pth vqa_last.pth

Loading JSON data...
Building Vocabulary...
Vocab size: 1000
Splitting data...
Train size: 355005, Val size: 88752


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch 1/6 | Train Loss: 2.5254 | Train Acc: 0.3180 | Val Acc: 0.3629
saved last model
 -> Saved best model.
  adding: vqa_best.pth (deflated 7%)
  adding: vqa_last.pth (deflated 7%)


Training:   0%|          | 0/11094 [00:00<?, ?it/s]

Validating:   0%|          | 0/2774 [00:00<?, ?it/s]

Epoch 2/6 | Train Loss: 2.0148 | Train Acc: 0.3642 | Val Acc: 0.3790
saved last model
 -> Saved best model.
updating: vqa_best.pth (deflated 7%)
updating: vqa_last.pth (deflated 7%)


Training:   0%|          | 0/11094 [00:00<?, ?it/s]

In [ ]:
import os
print(os.listdir("/kaggle/working"))
print(os.listdir("/kaggle/working/.virtual_documents"))

In [4]:
!zip model.zip vqa_best.pth vqa_last.pth


	zip warning: name not matched: vqa_best.pth
	zip warning: name not matched: vqa_last.pth

zip error: Nothing to do! (model.zip)


In [ ]:
import json
from IPython.display import FileLink

# 1. Save the file locally in the kernel
with open('vocab.json', 'w') as f:
    json.dump(answer_to_idx, f)

# 2. Generate a download link
FileLink('vocab.json')